In [1]:
# GENERAL THINGS FOR COMPUTING AND PLOTTING
import pandas as pd
import numpy as np
import os, sys, time
from datetime import datetime
from datetime import timedelta
import scipy as sp

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", context="talk")

# ibl specific things
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior
from ibl_pipeline.analyses import behavior as behavioral_analyses


Connecting anneurai@datajoint.internationalbrainlab.org:3306


In [2]:
# all mice that started training
# leave out mice in the brain-wide project that got too old before the ephys setup was ready...
all_mice = (subject.Subject & 'subject_birth_date > "2019-03-01"') \
    * (subject.SubjectProject & 'subject_project = "ibl_neuropixel_brainwide_01"') 
    #* (subject.SubjectLab & 'lab_name = "churchlandlab"')
all_mice = all_mice.fetch(format='frame').reset_index()
print('# of animals in brainwide project:')
all_mice.subject_nickname.nunique()


# of animals in brainwide project:


128

In [10]:
# all mice that made it to ready4ephys
all_ephys_sess = (subject.Subject & 'subject_birth_date > "2019-03-01"') \
    * (subject.SubjectProject & 'subject_project = "ibl_neuropixel_brainwide_01"') \
    * (acquisition.Session & 'task_protocol LIKE "%ephysChoice%"')
  #  * (subject.SubjectLab & 'lab_name = "churchlandlab"') \

all_ephys_sess = all_ephys_sess.fetch(format='frame').reset_index()
print('# of animals with ephys sessions:')
all_ephys_sess.subject_nickname.nunique()


# of animals with ephys sessions:


42

In [11]:
animals_noephys = list(set(all_mice.subject_nickname.unique()) - set(all_ephys_sess.subject_nickname.unique()))
print('animals without any ephys data:')
sorted(animals_noephys)

animals without any ephys data:


['CSHL046',
 'CSHL048',
 'CSHL056',
 'CSHL057',
 'CSHL058',
 'CSH_ZAD_002',
 'CSH_ZAD_003',
 'CSH_ZAD_004',
 'CSH_ZAD_005',
 'CSH_ZAD_006',
 'CSH_ZAD_007',
 'CSH_ZAD_008',
 'CSH_ZAD_009',
 'CSH_ZAD_010',
 'CSH_ZAD_015',
 'CSH_ZAD_016',
 'CSH_ZAD_017',
 'CSH_ZAD_018',
 'CSH_ZAD_019',
 'CSH_ZAD_021',
 'CSH_ZAD_022',
 'CSH_ZAD_024',
 'CSH_ZAD_23',
 'DY_012',
 'DY_014',
 'DY_015',
 'KS017',
 'KS019',
 'KS021',
 'KS024',
 'KS025',
 'KS030',
 'KS031',
 'KS032',
 'KS033',
 'KS034',
 'KS035',
 'NYU-21',
 'NYU-22',
 'NYU-23',
 'NYU-24',
 'NYU-25',
 'NYU-26',
 'SWC_006',
 'SWC_007',
 'SWC_010',
 'SWC_011',
 'SWC_016',
 'SWC_017',
 'SWC_019',
 'SWC_020',
 'SWC_022',
 'SWC_027',
 'SWC_028',
 'SWC_029',
 'SWC_030',
 'SWC_032',
 'SWC_033',
 'SWC_034',
 'SWC_035',
 'SWC_036',
 'SWC_038',
 'SWC_039',
 'SWC_040',
 'SWC_041',
 'SWC_042',
 'SWC_043',
 'ZM_1743',
 'ZM_3001',
 'ZM_3002',
 'ZM_3003',
 'ZM_3004',
 'ZM_3005',
 'ZM_3006',
 'ibl_witten_11',
 'ibl_witten_12',
 'ibl_witten_15',
 'ibl_witten_16',


#### animals that never made it to ephys 

* CSHL048: issue during headbar implant surgery, never started training
* CSHL056: experimental well from tip of centrifuge tube, infection on skull
* CSHL057: experimental well from tip of centrifuge tube, infection on skull

* CSHL046: has not reached ready4ephysRig, now > 7 months
* CSHL058: now on biasedCW, will hopefully still record


In [12]:
### for those mice with ephys data, how many sessions?
print('average number of sessions per mouse:')
print(all_ephys_sess.groupby(['subject_nickname'])['session_start_time'].nunique().mean())

all_ephys_sess.groupby(['subject_nickname'])['session_start_time'].nunique().reset_index().sort_values(by='subject_nickname')

average number of sessions per mouse:
5.023809523809524


,subject_nickname,session_start_time
0,CSHL045,6
1,CSHL047,6
2,CSHL049,8
3,CSHL051,4
4,CSHL052,6
5,CSHL053,6
6,CSHL054,2
7,CSHL055,2
8,CSHL059,5
9,CSHL060,4


### cull/ephys end reasons

##### bad animals
* CSHL054: session 1 great. session 2 terrible behavior, mouse looks like it's in pain
* CSHL055 (honeycomb): session 1 great, session 2 awful. mouse lost weight, lethargic, died 4 days after craniotomy surgery
* CSHL051: when attempting to do second cranio surgery with punch, skull broke. emergency perfusion.

##### good animals
* CSHL059: no more sites to go into the brain
* CSHL045: 
* CSHL047:
* CSHL052: honeycomb; quite a lot of blood in cranio but behavior great.
* CSHL053: 
* CSHL049: 
* CSHL060: still ingesting (has 6 sessions)


In [20]:
# how many probes per ephys session?
ephys = dj.create_virtual_module('ephys', 'ibl_ephys')

all_ephys_sess = (subject.Subject & 'subject_birth_date > "2019-03-01"') \
    * (subject.SubjectProject & 'subject_project = "ibl_neuropixel_brainwide_01"') \
    * (acquisition.Session & 'task_protocol LIKE "%ephysChoice%"') \
    * ephys.ProbeInsertion * (ephys.ProbeTrajectory & 'insertion_data_source = "Micro-manipulator"') \
    * subject.SubjectLab()
all_ephys_sess = all_ephys_sess.fetch(format='frame').reset_index()

In [21]:
### for those mice with ephys data, how many sessions?
yield_permouse = all_ephys_sess.groupby(['lab_name', 'subject_nickname'])['session_start_time', 'probe_trajectory_uuid'].nunique().reset_index()
yield_permouse.rename(columns={'session_start_time':'num_sessions', 'probe_trajectory_uuid':'num_penetrations'}, inplace=True)
assert ((yield_permouse['num_sessions'] * 2 >= yield_permouse['num_penetrations']).all() == True)

print('average number of penetrations per mouse:')
print(yield_permouse['num_penetrations'].mean())

yield_permouse.sort_values(by='subject_nickname')

average number of penetrations per mouse:
6.607142857142857


,lab_name,subject_nickname,num_sessions,num_penetrations
1,churchlandlab,CSHL045,4,7
2,churchlandlab,CSHL047,5,9
3,churchlandlab,CSHL049,6,9
4,churchlandlab,CSHL051,4,8
5,churchlandlab,CSHL052,5,8
6,churchlandlab,CSHL053,4,7
7,churchlandlab,CSHL054,2,3
8,churchlandlab,CSHL055,2,4
9,churchlandlab,CSHL059,5,9
10,churchlandlab,CSHL060,3,6


In [22]:
yield_permouse.groupby(['lab_name'])['num_penetrations'].describe()

,count,mean,std,min,25%,50%,75%,max
lab_name,,,,,,,,
angelakilab,1.0,8.000000,NaN,8.0,8.00,8.0,8.00,8.0
churchlandlab,10.0,7.000000,2.108185,3.0,6.25,7.5,8.75,9.0
cortexlab,6.0,6.833333,3.311596,1.0,6.00,7.5,9.00,10.0
danlab,1.0,12.000000,NaN,12.0,12.00,12.0,12.00,12.0
hoferlab,3.0,5.000000,5.196152,2.0,2.00,2.0,6.50,11.0
mainenlab,3.0,5.000000,2.645751,2.0,4.00,6.0,6.50,7.0
mrsicflogellab,2.0,7.000000,1.414214,6.0,6.50,7.0,7.50,8.0
wittenlab,1.0,4.000000,NaN,4.0,4.00,4.0,4.00,4.0
zadorlab,1.0,6.000000,NaN,6.0,6.00,6.0,6.00,6.0
